In [2]:
import datetime
import tarfile
import json
import bz2
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [13]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf, desc, min, max, to_date, date_format, col, expr, hour, year, month, dayofweek, count
from pyspark.sql import functions as F #module that includes a variety of functions like to extract features

In [4]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### Loading csv data file to pyspark

In [62]:
# defining schema per column, header and data type
schema = StructType([
    StructField("index",IntegerType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("date", DateType(), nullable=True),
    StructField("flag", StringType(), nullable=True),
    StructField("user", StringType(), nullable=True),
    StructField("text", StringType(), nullable=True)
])

#reading in the file with the schema defined above
tweetsDf= spark.read.csv('hdfs://localhost:9000/user1/ProjectTweets.csv', schema=schema, header=False).withColumn("date", to_date("date"))

#printing schema and head of the table
tweetsDf.printSchema();tweetsDf.show()

root
 |-- index: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)

+-----+----------+----+--------+---------------+--------------------+
|index|        id|date|    flag|           user|                text|
+-----+----------+----+--------+---------------+--------------------+
|    0|1467810369|null|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|    1|1467810672|null|NO_QUERY|  scotthamilton|is upset that he ...|
|    2|1467810917|null|NO_QUERY|       mattycus|@Kenichan I dived...|
|    3|1467811184|null|NO_QUERY|        ElleCTF|my whole body fee...|
|    4|1467811193|null|NO_QUERY|         Karoli|@nationwideclass ...|
|    5|1467811372|null|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|    6|1467811592|null|NO_QUERY|        mybirch|         Need a hug |
|    7|1467811594|null|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|    8|14

checking flag column

In [8]:
#extracting "flag" column as RDD
flag_rdd = tweetsDf.select("flag").rdd

# getting distinct values
unique_flags = flag_rdd.distinct()

#collecting the distinct values
unique_flags.collect()

[Row(flag='NO_QUERY')]

In [61]:
# ADDressing PDT issue

from pyspark.sql.functions import regexp_replace,


schema = StructType([
    StructField("index", StringType(), True),
    StructField("id", StringType(), True),
    StructField("date", StringType(), True),
    StructField("flag", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
])

tweetsDf= spark.read.csv('hdfs://localhost:9000/user1/ProjectTweets.csv', schema=schema, header=False)

tweetsDf = tweetsDf.withColumn("date", regexp_replace(col("date"), "PDT", "UTC"))

tweetsDf = tweetsDf.withColumn("date", to_timestamp("date", "EEE MMM dd HH:mm:ss zzz yyyy").cast(DateType()))

tweetsDf.printSchema();tweetsDf.show()


root
 |-- index: string (nullable = true)
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



Py4JJavaError: An error occurred while calling o419.showString.
: org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to recognize 'EEE MMM dd HH:mm:ss zzz yyyy' pattern in the DateTimeFormatter. 1) You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0. 2) You can form a valid datetime pattern with the guide from https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToRecognizePatternAfterUpgradeError(QueryExecutionErrors.scala:936)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkLegacyFormatter$1.applyOrElse(DateTimeFormatterHelper.scala:187)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkLegacyFormatter$1.applyOrElse(DateTimeFormatterHelper.scala:180)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql

In [43]:
# Read the CSV file, assuming it has no header and the columns are tab-separated
columns = ["index", "id", "date", "flag", "user", "text"]
pdDf = pd.read_csv("ProjectTweets.csv", header=None, names=columns)

pdDf

,index,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1599995,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1599996,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1599997,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1599998,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [31]:
pdDf

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,1599995,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,1599996,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,1599997,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,1599998,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [49]:
df['date'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [63]:
df = pd.read_csv('ProjectTweets.csv',header=None, names=columns)

# Convert the "date" column to date format without specifying format
df['date'] = pd.to_datetime(df['date'], errors='coerce', infer_datetime_format=True)

df

/usr/lib/python3/dist-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,index,id,date,flag,user,text
0,0,1467810369,2009-04-06 22:19:45,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1599995,2193601966,2009-06-16 08:40:49,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1599996,2193601969,2009-06-16 08:40:49,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1599997,2193601991,2009-06-16 08:40:49,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1599998,2193602064,2009-06-16 08:40:49,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [72]:
df.drop('index',axis=1)

,id,date,flag,user,text
0,1467810369,2009-04-06 22:19:45,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...
1599995,2193601966,2009-06-16 08:40:49,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,2193601969,2009-06-16 08:40:49,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,2193601991,2009-06-16 08:40:49,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,2193602064,2009-06-16 08:40:49,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [64]:
len(pdDf)==len(df)

True

In [74]:
df.drop('index',axis=1).to_csv('ProjectTweetsDates2.csv')

In [75]:
#tweetsDf.drop()

In [81]:
# defining schema per column, header and data type
schema = StructType([
    StructField("index",IntegerType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("date", DateType(), nullable=True),
    StructField("flag", StringType(), nullable=True),
    StructField("user", StringType(), nullable=True),
    StructField("text", StringType(), nullable=True)
])

#reading in the file with the schema defined above
tweetsDf= spark.read.csv('hdfs://localhost:9000/user1/ProjectTweetsDates2.csv', schema=schema, header=False).withColumn("date", to_date("date"))

#printing schema and head of the table
tweetsDf.printSchema();tweetsDf.show()

root
 |-- index: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)

+-----+----------+----------+--------+---------------+--------------------+
|index|        id|      date|    flag|           user|                text|
+-----+----------+----------+--------+---------------+--------------------+
| null|      null|      null|    flag|           user|                text|
|    0|1467810369|2009-04-06|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|    1|1467810672|2009-04-06|NO_QUERY|  scotthamilton|is upset that he ...|
|    2|1467810917|2009-04-06|NO_QUERY|       mattycus|@Kenichan I dived...|
|    3|1467811184|2009-04-06|NO_QUERY|        ElleCTF|my whole body fee...|
|    4|1467811193|2009-04-06|NO_QUERY|         Karoli|@nationwideclass ...|
|    5|1467811372|2009-04-06|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|    6|1467811592|2

In [86]:
tweetsDf.show(1)

+-----+----+----+----+----+----+
|index|  id|date|flag|user|text|
+-----+----+----+----+----+----+
| null|null|null|flag|user|text|
+-----+----+----+----+----+----+
only showing top 1 row



In [92]:
tweetsDf = tweetsDf.drop("null")

In [93]:
tweetsDf.show()

+-----+----------+----------+--------+---------------+--------------------+
|index|        id|      date|    flag|           user|                text|
+-----+----------+----------+--------+---------------+--------------------+
| null|      null|      null|    flag|           user|                text|
|    0|1467810369|2009-04-06|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|    1|1467810672|2009-04-06|NO_QUERY|  scotthamilton|is upset that he ...|
|    2|1467810917|2009-04-06|NO_QUERY|       mattycus|@Kenichan I dived...|
|    3|1467811184|2009-04-06|NO_QUERY|        ElleCTF|my whole body fee...|
|    4|1467811193|2009-04-06|NO_QUERY|         Karoli|@nationwideclass ...|
|    5|1467811372|2009-04-06|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|    6|1467811592|2009-04-06|NO_QUERY|        mybirch|         Need a hug |
|    7|1467811594|2009-04-06|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|    8|1467811795|2009-04-06|NO_QUERY|2Hood4Hollywood|@Tatiana_K nope t...|
|    9|14678